In [1]:
import os 
from model import AfrikaPOS

/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
LANG = 'ewe'
DATA_DIR = os.path.join('..', 'data_source', 'masakhane-pos', 'data', LANG)
BASE_MODEL = 'bert-base-uncased'

NUM_HIDDENS = 2
NUM_ATTENTIONS = 1

In [3]:
model = AfrikaPOS(DATA_DIR, NUM_HIDDENS, NUM_ATTENTIONS, BASE_MODEL)




huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Map: 100%|██████████| 145/145 [00:00<00:00, 17509.98 examples/s]


In [4]:
model.train_model(f"./models/{LANG}", num_train_epochs=10)

 10%|▉         | 45/460 [00:05<00:43,  9.49it/s]/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control th

{'eval_loss': 0.597817599773407, 'eval_precision': 0.6085235545373097, 'eval_recall': 0.6273938384679434, 'eval_f1': 0.5830292259512614, 'eval_accuracy': 0.6273938384679434, 'eval_runtime': 0.5057, 'eval_samples_per_second': 286.748, 'eval_steps_per_second': 19.776, 'epoch': 1.0}


 20%|█▉        | 91/460 [00:09<00:25, 14.68it/s]/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control th

{'eval_loss': 0.44979915022850037, 'eval_precision': 0.7331041688166248, 'eval_recall': 0.7119067443796836, 'eval_f1': 0.6893820737755906, 'eval_accuracy': 0.7119067443796836, 'eval_runtime': 0.1561, 'eval_samples_per_second': 928.618, 'eval_steps_per_second': 64.043, 'epoch': 2.0}


 30%|██▉       | 137/460 [00:12<00:22, 14.21it/s]/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control t

...

{'eval_loss': 0.30380529165267944, 'eval_precision': 0.8104234932662594, 'eval_recall': 0.8051623646960866, 'eval_f1': 0.7982095677113112, 'eval_accuracy': 0.8051623646960866, 'eval_runtime': 0.151, 'eval_samples_per_second': 960.539, 'eval_steps_per_second': 66.244, 'epoch': 9.0}


100%|█████████▉| 459/460 [00:38<00:00, 14.95it/s]/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control t

{'eval_loss': 0.30298858880996704, 'eval_precision': 0.8110532581043788, 'eval_recall': 0.8068276436303081, 'eval_f1': 0.8005439967759289, 'eval_accuracy': 0.8068276436303081, 'eval_runtime': 0.1516, 'eval_samples_per_second': 956.533, 'eval_steps_per_second': 65.968, 'epoch': 10.0}


100%|██████████| 460/460 [00:39<00:00, 11.75it/s]

{'train_runtime': 39.1646, 'train_samples_per_second': 185.882, 'train_steps_per_second': 11.745, 'train_loss': 0.4611353335173234, 'epoch': 10.0}


In [5]:
model.evaluate_on_test()

/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

,ADJ,ADP,ADV,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,accuracy,macro avg,weighted avg
precision,0.679688,0.857857,0.617647,0.994083,0.902062,0.863636,0.716787,0.809091,0.991453,0.869388,0.746936,0.953728,0.806167,0.0,0.578947,0.784857,0.759165,0.787044
recall,0.334615,0.818760,0.244186,0.861538,0.879397,1.000000,0.863273,0.328413,0.840580,0.516990,0.734752,0.997312,0.890511,0.0,0.617284,0.784857,0.661841,0.784857
f1-score,0.448454,0.837852,0.350000,0.923077,0.890585,0.926829,0.783240,0.467192,0.909804,0.648402,0.740794,0.975033,0.846243,0.0,0.597501,0.784857,0.689667,0.777207
support,260.000000,1887.000000,86.000000,195.000000,597.000000,38.000000,2750.000000,271.000000,138.000000,412.000000,1410.000000,1116.000000,411.000000,38.0,891.000000,0.784857,10500.000000,10500.000000
